# Best Model segun error absoluto de la predicción de 201912

In [1]:
# from google.colab import drive
import pandas as pd
import os


#  Definir rutas en Drive
BASE_INTERMEDIOS   = '/Users/indianaabeledo/Documents/Maestria/laboratorio_III/labo3-2025v/entrega_final/intermedios/'
BASE_OUTPUTS   = '/Users/indianaabeledo/Documents/Maestria/laboratorio_III/labo3-2025v/entrega_final/output/'

os.makedirs(BASE_OUTPUTS, exist_ok=True)

# Carga de archivos desde Drive
print("🔄 Cargando archivos desde Drive...")
#pred_autogluon = pd.read_csv(os.path.join(BASE_OUTPUTS, "pred_modelo_autogluon.csv"))
pred_autogluon = pd.read_csv(os.path.join(BASE_OUTPUTS, "predicciones_modelo_Autogluonv2 (2).csv"))
pred_lgbm = pd.read_csv(os.path.join(BASE_OUTPUTS, "pred_modelo_lightgbm_best.csv"))
pred_RL= pd.read_csv(os.path.join(BASE_OUTPUTS, "pred_modelo_regresion_lineal.csv"))

error_autogluon = pd.read_csv(os.path.join(BASE_OUTPUTS, "error_201912_autogluon.csv"))
error_lgbm = pd.read_csv(os.path.join(BASE_OUTPUTS, "error_201912_lightgbm.csv"))
error_RL = pd.read_csv(os.path.join(BASE_OUTPUTS, "error_201912_regresion_lineal.csv"))

df_limpio= pd.read_csv(os.path.join(BASE_INTERMEDIOS, "df_limpio_product_id.csv"))

# Renames
pred_autogluon.rename(columns={'tn': 'tn_pred'}, inplace=True)
error_lgbm = error_lgbm.rename(columns={'target': 'tn_real'})  # renombrar 'tn' a 'tn_real'
# seleccionar solo las columnas deseadas
error_lgbm = error_lgbm[['product_id', 'tn_real', 'tn_pred', 'abs_error', 'rel_error']]

🔄 Cargando archivos desde Drive...


### Calculamos el promedio de la tn por product_id de los últimos 12 meses para 201912
Sera otra de las opciones de predicción

In [2]:
# Calcular promedio tn de los últimos 12 meses disponibles por product_id==201912
ultimos_meses = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911,201912]
pred_prom_dic19_tn_12M = (
    df_limpio[df_limpio['periodo'].isin(ultimos_meses)]
      .groupby('product_id', as_index=False)
      .agg(avg=('tn','mean'))

)
pred_prom_dic19_tn_12M.head()

,product_id,avg
0,20001,1454.732692
1,20002,1175.437148
2,20003,784.976412
3,20004,627.215330
4,20005,668.270122


### Calculamos el promedio de la tn por product_id de los últimos 12 meses para 201910
Con el objetivo de calcular el error absoluto con la tn real

In [3]:
# Calcular promedio tn de los últimos 12 meses disponibles por product_id==201910
ultimos_meses = [201811, 201812,201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910]
pred_prom_oct19_tn_12M = (
    df_limpio[df_limpio['periodo'].isin(ultimos_meses)]
      .groupby('product_id', as_index=False)
      .agg(avg=('tn','mean'))

)

pred_prom_oct19_tn_12M.head()

,product_id,avg
0,20001,1487.869425
1,20002,1197.552089
2,20003,796.305674
3,20004,629.387777
4,20005,638.415252


In [4]:
df_avg=df_limpio[['product_id','tn']][df_limpio['periodo']==201910]
df_avg.head()

,product_id,tn
33,20001,1561.5055
69,20002,1979.5364
105,20003,1081.3665
141,20004,1064.6963
177,20005,996.7828


### Calculamos el error absoluto entre el promedio de los últimos 12 meses para 201912 y la tn real


In [5]:
# Average: Merge entre tn y avg oct 2019
df_avg= pd.merge(df_avg,pred_prom_oct19_tn_12M, on='product_id', how='left')
df_avg.head()

,product_id,tn,avg
0,20001,1561.5055,1487.869425
1,20002,1979.5364,1197.552089
2,20003,1081.3665,796.305674
3,20004,1064.6963,629.387777
4,20005,996.7828,638.415252


In [6]:
# Calculo de error 201910
df_avg['abs_error'] = (df_avg['tn'] - df_avg['avg']).abs()
df_avg.rename(columns={'avg': 'tn_pred'}, inplace=True)
df_avg.head()

,product_id,tn,tn_pred,abs_error
0,20001,1561.5055,1487.869425,73.636075
1,20002,1979.5364,1197.552089,781.984311
2,20003,1081.3665,796.305674,285.060826
3,20004,1064.6963,629.387777,435.308523
4,20005,996.7828,638.415252,358.367548


### Unimos los errores absolutos de todos los modelos

In [7]:
# Merge por product_id y agregar sufijos para columnas comunes
error_merge = pd.merge(
    error_lgbm,
    error_autogluon,
    on='product_id',
    suffixes=('_lgbm', '_auto')
)
error_merge.head()

,product_id,tn_real_lgbm,tn_pred_lgbm,abs_error_lgbm,rel_error,tn_real_auto,tn_pred_auto,abs_error_auto,pct_error,weight
0,20001,1504.68860,1072.377714,432.310884,0.287309,1504.68856,1607.053592,102.365032,0.068031,0.059840
1,20012,173.13004,334.821354,161.691319,0.933930,173.13004,325.928112,152.798072,0.882562,0.006885
2,20008,195.36855,347.529457,152.160911,0.778840,195.36854,430.347867,234.979327,1.202749,0.007770
3,20014,272.02810,390.193327,118.165220,0.434386,272.02812,441.499765,169.471645,0.622993,0.010818
4,20023,181.13277,283.990365,102.857599,0.567857,181.13277,305.751655,124.618885,0.687997,0.007203


In [8]:
error_merge_final = pd.merge(
    error_merge,
    error_RL,
    on='product_id')

error_merge_final.head()

,product_id,tn_real_lgbm,tn_pred_lgbm,abs_error_lgbm,rel_error,tn_real_auto,tn_pred_auto,abs_error_auto,pct_error,weight,tn_pred_RL,abs_error_RL,rel_error_RL
0,20001,1504.68860,1072.377714,432.310884,0.287309,1504.68856,1607.053592,102.365032,0.068031,0.059840,1260.707460,300.798060,0.192633
1,20012,173.13004,334.821354,161.691319,0.933930,173.13004,325.928112,152.798072,0.882562,0.006885,295.419673,13.120203,0.046476
2,20008,195.36855,347.529457,152.160911,0.778840,195.36854,430.347867,234.979327,1.202749,0.007770,281.057816,171.714154,0.379251
3,20014,272.02810,390.193327,118.165220,0.434386,272.02812,441.499765,169.471645,0.622993,0.010818,322.422187,189.583853,0.370277
4,20023,181.13277,283.990365,102.857599,0.567857,181.13277,305.751655,124.618885,0.687997,0.007203,234.179244,152.200386,0.393914


In [9]:
# Partimos de tu df original
df = error_merge_final.copy()

# Lista de columnas de error
error_cols = ['abs_error_lgbm', 'abs_error_auto', 'abs_error_RL']

# 1) Calculamos para cada producto cuál columna tiene el valor mínimo
best_idx = df.set_index('product_id')[error_cols].idxmin(axis=1)

# 2) Extraemos el nombre del modelo a partir del sufijo de la columna
best_model = best_idx.str.replace('abs_error_', '')

# 3) Opcional: también recuperar el valor mínimo de error
min_error = df.set_index('product_id')[error_cols].min(axis=1)

# 4) Construimos el DataFrame resultante
df_best = pd.DataFrame({
    'product_id': best_model.index,
    'best_model': best_model.values,
    'min_abs_error': min_error.values
}).reset_index(drop=True)

print(df_best.head())

   product_id best_model  min_abs_error
0       20001       auto     102.365032
1       20012         RL      13.120203
2       20008       lgbm     152.160911
3       20014       lgbm     118.165220
4       20023       lgbm     102.857599


In [10]:
pred_lgbm.rename(columns={'tn_pred': 'lgbm'}, inplace=True)
pred_autogluon.rename(columns={'tn_pred': 'auto'}, inplace=True)
pred_RL.rename(columns={'pred': 'RL'}, inplace=True)

In [11]:
merge_pred= pd.merge(pred_autogluon, pred_lgbm, on='product_id')
merge_pred= pd.merge(merge_pred, pred_RL, on='product_id')
merge_pred.shape[0]

780

In [12]:
merge_pred.head()

,product_id,auto,lgbm,RL
0,20001,1318.842610,1072.377714,1162.707525
1,20002,1088.935465,1078.858007,1183.640604
2,20003,718.753417,835.263464,684.763931
3,20004,546.314163,585.657638,580.484961
4,20005,529.565386,522.645068,563.560780


In [13]:
merge_pred_final= pd.merge(merge_pred, df_best, on='product_id')
merge_pred_final.shape[0]

780

In [14]:
merge_pred_final.head()

,product_id,auto,lgbm,RL,best_model,min_abs_error
0,20001,1318.842610,1072.377714,1162.707525,auto,102.365032
1,20002,1088.935465,1078.858007,1183.640604,lgbm,8.450587
2,20003,718.753417,835.263464,684.763931,lgbm,57.237818
3,20004,546.314163,585.657638,580.484961,lgbm,52.242387
4,20005,529.565386,522.645068,563.560780,lgbm,70.599378


In [15]:
merge_pred_final.to_csv(os.path.join(BASE_OUTPUTS, "merge_pred_final.csv"), index=False)

In [16]:
# Partimos de tu merge_pred_final
df = merge_pred_final.copy()

# 1) Definimos el mapeo entre el valor de best_model y la columna de predicción
col_map = {
    'auto':    'auto',
    'lgbm':    'lgbm',
    'RL':      'RL'
}

# 2) Creamos el nuevo DF con product_id y la predicción del mejor modelo
df_best_pred = pd.DataFrame({
    'product_id': df['product_id'],
    'best_pred':  df.apply(lambda row: row[col_map[row['best_model']]], axis=1)
})

df_best_pred.head()

,product_id,best_pred
0,20001,1318.842610
1,20002,1078.858007
2,20003,835.263464
3,20004,585.657638
4,20005,522.645068


In [17]:
df_best_pred.to_csv(os.path.join(BASE_OUTPUTS, "best_model_product_id.csv"), index=False)

# Agregamos Promedio al Merge de RL/Auto/LGBM

In [18]:
df_final_avg = pd.merge(
    error_merge_final,
    df_avg,
    on='product_id')
df_final_avg.rename(columns={'abs_error': 'abs_error_avg'}, inplace=True)
df_final_avg.head()

,product_id,tn_real_lgbm,tn_pred_lgbm,abs_error_lgbm,rel_error,tn_real_auto,tn_pred_auto,abs_error_auto,pct_error,weight,tn_pred_RL,abs_error_RL,rel_error_RL,tn,tn_pred,abs_error_avg
0,20001,1504.68860,1072.377714,432.310884,0.287309,1504.68856,1607.053592,102.365032,0.068031,0.059840,1260.707460,300.798060,0.192633,1561.50550,1487.869425,73.636075
1,20012,173.13004,334.821354,161.691319,0.933930,173.13004,325.928112,152.798072,0.882562,0.006885,295.419673,13.120203,0.046476,282.29947,343.879977,61.580508
2,20008,195.36855,347.529457,152.160911,0.778840,195.36854,430.347867,234.979327,1.202749,0.007770,281.057816,171.714154,0.379251,452.77197,444.378500,8.393470
3,20014,272.02810,390.193327,118.165220,0.434386,272.02812,441.499765,169.471645,0.622993,0.010818,322.422187,189.583853,0.370277,512.00604,434.283848,77.722192
4,20023,181.13277,283.990365,102.857599,0.567857,181.13277,305.751655,124.618885,0.687997,0.007203,234.179244,152.200386,0.393914,386.37964,287.076334,99.303306


In [19]:
# Lista de columnas de error
error_cols = ['abs_error_lgbm', 'abs_error_auto', 'abs_error_RL', 'abs_error_avg']

# 1) Calculamos para cada producto cuál columna tiene el valor mínimo
best_idx_2 = df_final_avg.set_index('product_id')[error_cols].idxmin(axis=1)

# 2) Extraemos el nombre del modelo a partir del sufijo de la columna
best_model_2= best_idx_2.str.replace('abs_error_', '')

# 3) Opcional: también recuperar el valor mínimo de error
min_error_2 = df_final_avg.set_index('product_id')[error_cols].min(axis=1)

# 4) Construimos el DataFrame resultante
df_best_with_avg = pd.DataFrame({
    'product_id': best_model_2.index,
    'best_model': best_model_2.values,
    'min_abs_error': min_error_2.values
}).reset_index(drop=True)

print(df_best_with_avg.head())

   product_id best_model  min_abs_error
0       20001        avg      73.636075
1       20012         RL      13.120203
2       20008        avg       8.393470
3       20014        avg      77.722192
4       20023        avg      99.303306


In [20]:
df_best_with_avg['best_model'].value_counts(normalize=True)

best_model
avg     0.319231
auto    0.269231
lgbm    0.217949
RL      0.193590
Name: proportion, dtype: float64

In [21]:
pred_lgbm.rename(columns={'tn_pred': 'lgbm'}, inplace=True)
pred_autogluon.rename(columns={'tn_pred': 'auto'}, inplace=True)
pred_RL.rename(columns={'pred': 'RL'}, inplace=True)

merge_pred_2= pd.merge(pred_autogluon, pred_lgbm, on='product_id')
merge_pred_2= pd.merge(merge_pred_2, pred_RL, on='product_id')
merge_pred_2= pd.merge(merge_pred_2,pred_prom_dic19_tn_12M, on='product_id')
merge_pred_2.head()

,product_id,auto,lgbm,RL,avg
0,20001,1318.842610,1072.377714,1162.707525,1454.732692
1,20002,1088.935465,1078.858007,1183.640604,1175.437148
2,20003,718.753417,835.263464,684.763931,784.976412
3,20004,546.314163,585.657638,580.484961,627.215330
4,20005,529.565386,522.645068,563.560780,668.270122


In [22]:
merge_pred_final_2= pd.merge(merge_pred_2, df_best_with_avg, on='product_id')
merge_pred_final_2.head()

,product_id,auto,lgbm,RL,avg,best_model,min_abs_error
0,20001,1318.842610,1072.377714,1162.707525,1454.732692,avg,73.636075
1,20002,1088.935465,1078.858007,1183.640604,1175.437148,lgbm,8.450587
2,20003,718.753417,835.263464,684.763931,784.976412,lgbm,57.237818
3,20004,546.314163,585.657638,580.484961,627.215330,lgbm,52.242387
4,20005,529.565386,522.645068,563.560780,668.270122,lgbm,70.599378


In [23]:
# 1) Definimos el mapeo entre el valor de best_model y la columna de predicción
col_map = {
    'auto':    'auto',
    'lgbm':    'lgbm',
    'RL':      'RL',
    'avg':    'avg'
}

# 2) Creamos el nuevo DF con product_id y la predicción del mejor modelo
df_best_pred_2 = pd.DataFrame({
    'product_id': merge_pred_final_2['product_id'],
    'best_pred':  merge_pred_final_2.apply(lambda row: row[col_map[row['best_model']]], axis=1)
})

df_best_pred_2.head()

,product_id,best_pred
0,20001,1454.732692
1,20002,1078.858007
2,20003,835.263464
3,20004,585.657638
4,20005,522.645068


In [24]:
df_best_pred_2.to_csv(os.path.join(BASE_OUTPUTS, "best_model_product_id_with_avg.csv"), index=False)

In [27]:
merge_pred_final_2.head()

,product_id,auto,lgbm,RL,avg,best_model,min_abs_error
0,20001,1318.842610,1072.377714,1162.707525,1454.732692,avg,73.636075
1,20002,1088.935465,1078.858007,1183.640604,1175.437148,lgbm,8.450587
2,20003,718.753417,835.263464,684.763931,784.976412,lgbm,57.237818
3,20004,546.314163,585.657638,580.484961,627.215330,lgbm,52.242387
4,20005,529.565386,522.645068,563.560780,668.270122,lgbm,70.599378


In [28]:
# Partimos de tu DataFrame original
prediccion_ponderada = merge_pred_final_2[['product_id','auto','lgbm','RL','avg']].copy()

# 1) Creamos la columna 'pred' como la media de las 4 columnas
prediccion_ponderada['pred'] = prediccion_ponderada[
    ['auto','lgbm','RL','avg']
].mean(axis=1)

# 2) Eliminamos las columnas originales
prediccion_ponderada = prediccion_ponderada.drop(
    ['auto','lgbm','RL','avg'],
    axis=1
)

# Ahora prediccion_ponderada sólo tiene product_id y pred
print(prediccion_ponderada.head())

   product_id         pred
0       20001  1252.165135
1       20002  1131.717806
2       20003   755.939306
3       20004   584.918023
4       20005   571.010339


In [29]:
prediccion_ponderada.to_csv(os.path.join(BASE_OUTPUTS, "best_model_product_id_pond_final.csv"), index=False)